In [1]:
import geopandas as gpd
import pandas as pd


In [2]:
meta_data = pd.read_csv(
    '/resnick/groups/enceladus/glavrent/Scalable_GPs/Data/preprocessing/gm_metadata.csv'
)
meta_data = meta_data.head(100)

In [3]:
meta_data_gdf = gpd.GeoDataFrame(
    meta_data,
    geometry=gpd.points_from_xy(meta_data['site_lon'], meta_data['site_lat']),
    crs="EPSG:4326"
)
meta_data_gdf.explore()

In [4]:
meta_data.utm_zone.unique()

array(['11S'], dtype=object)

In [5]:
meta_data_gdf[['site_x', 'site_y']]

,site_x,site_y
0,409.186696,3741.273790
1,437.161392,3762.230468
2,398.582546,3745.867561
3,404.986592,3770.495605
4,463.157015,3739.878958
...,...,...
95,442.841869,3720.365679
96,447.827633,3729.034495
97,452.813751,3737.702531
98,457.800181,3746.370920


In [6]:
meta_data_gdf_trans = meta_data_gdf.to_crs(epsg=32611)
meta_data_gdf_trans.geometry.x, meta_data_gdf_trans.geometry.y 

(0     409186.696023
 1     437161.392043
 2     398582.545724
 3     404986.592421
 4     463157.015143
           ...      
 95    442841.869441
 96    447827.633330
 97    452813.751054
 98    457800.181348
 99    462786.871936
 Length: 100, dtype: float64,
 0     3.741274e+06
 1     3.762230e+06
 2     3.745868e+06
 3     3.770496e+06
 4     3.739879e+06
           ...     
 95    3.720366e+06
 96    3.729034e+06
 97    3.737703e+06
 98    3.746371e+06
 99    3.755039e+06
 Length: 100, dtype: float64)

In [7]:
# Load the site_id metadata
sites_file = "/resnick/groups/enceladus/glavrent/Scalable_GPs/Raw_files/scec/study_22.12_sites.csv"
sites_df = pd.read_csv(sites_file)
sites_gdf = gpd.GeoDataFrame(
    sites_df,
    geometry=gpd.points_from_xy(sites_df['site_lon'], sites_df['site_lat']),
    crs="EPSG:4326" # As specified in the metadata
)
sites_gdf = sites_gdf.drop(columns = ['site_name',
                                        'model_vs30','class',
                                        'Z1.0', 'Z2.5'])
sites_gdf = sites_gdf.to_crs(epsg=32611) # Convert to the same CRS as all_data_gdf
all_data_gdf = gpd.sjoin_nearest(meta_data_gdf_trans, sites_gdf, how="left", distance_col="dist")

In [8]:
all_data_gdf.columns

Index(['scen_id', 'eq_id', 'source_id', 'rupture_id', 'site_id_left',
       'site_name', 'mag', 'dip', 'rake', 'z_tor', 'width', 'vs30_thompson',
       'vs30_scec', 'z1.0', 'z2.5', 'Rrup', 'Rfsrc', 'Rjb', 'Rx', 'Ry0',
       'utm_zone', 'source_mpt_lat', 'source_mpt_lon', 'source_mpt_x',
       'source_mpt_y', 'source_mpt_z', 'source_cst_lat', 'source_cst_lon',
       'source_cst_x', 'source_cst_y', 'source_cst_z', 'site_lat_left',
       'site_lon_left', 'site_x', 'site_y', 'geometry', 'index_right',
       'site_id_right', 'site_lat_right', 'site_lon_right', 'dist'],
      dtype='object')